## Reduce the size of large datasets by using the right formats

Pandas for large datasets -
https://www.dataquest.io/blog/pandas-big-data/

Other useful reads

* https://pythonspeed.com/articles/pandas-load-less-data/
* https://pythonspeed.com/articles/pandas-reduce-memory-lossy/
* https://pythonspeed.com/articles/chunking-pandas/
* https://pythonspeed.com/articles/faster-pandas-dask/

In [1]:
import pandas as pd
import numpy as np

In [2]:
btomb = 1024**2 # Byte to MBytes

In [4]:
df = pd.read_csv("/v/courses/dataexp2024.public/Datasets/D-LargeData/crimes.csv")


/tmp/ipykernel_135/2020463488.py:1: DtypeWarning: Columns (8,9,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/v/courses/dataexp2024.public/Datasets/D-LargeData/crimes.csv")


In [5]:
# That is the current space occupied by the data in the memory after loading it.
usage = df.memory_usage()
usage = usage / btomb
usage 

Index                   0.000122
ID                      3.434570
Case Number             3.434570
Date                    3.434570
Block                   3.434570
IUCR                    3.434570
Primary Type            3.434570
Description             3.434570
Location Description    3.434570
Arrest                  3.434570
Domestic                3.434570
Beat                    3.434570
District                3.434570
Ward                    3.434570
Community Area          3.434570
FBI Code                3.434570
X Coordinate            3.434570
Y Coordinate            3.434570
Year                    3.434570
Updated On              3.434570
Latitude                3.434570
Longitude               3.434570
Location                3.434570
dtype: float64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450176 entries, 0 to 450175
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   ID                    450176 non-null  int64  
 1   Case Number           450175 non-null  object 
 2   Date                  450176 non-null  object 
 3   Block                 450176 non-null  object 
 4   IUCR                  450176 non-null  object 
 5   Primary Type          450175 non-null  object 
 6   Description           450175 non-null  object 
 7   Location Description  446771 non-null  object 
 8   Arrest                450175 non-null  object 
 9   Domestic              450175 non-null  object 
 10  Beat                  450175 non-null  float64
 11  District              450175 non-null  float64
 12  Ward                  440258 non-null  float64
 13  Community Area        440337 non-null  float64
 14  FBI Code              450175 non-null  object 
 15  

In [7]:
print(np.iinfo('uint32'), np.iinfo('uint16'), df.ID.max())

Machine parameters for uint32
---------------------------------------------------------------
min = 0
max = 4294967295
---------------------------------------------------------------
 Machine parameters for uint16
---------------------------------------------------------------
min = 0
max = 65535
---------------------------------------------------------------
 12067449


In [8]:
#conv_ID = df.ID.apply(pd.to_numeric,downcast='unsigned')
conv_df = pd.DataFrame()
conv_df['ID'] = df.ID.astype("uint32")

In [9]:
print(f"Old dataframe: \n{df[['ID']].memory_usage() / btomb}\n New dataframe: {conv_df.memory_usage() / btomb}")

Old dataframe: 
Index    0.000122
ID       3.434570
dtype: float64
 New dataframe: Index    0.000122
ID       1.717285
dtype: float64


In [10]:
print(df.District.min(), df.District.max())

1.0 31.0


In [11]:
conv_df['District'] = df.District.astype("uint8")

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

There are NaNs. So first substitute them with an appropriate integer that will be possible to filter out later. 0 seems to be a good choice fo this

In [18]:
 df.loc[df.District.isna(), 'District'] = 0

In [19]:
conv_df['District'] = df.District.astype("uint8")

In [20]:
conv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450176 entries, 0 to 450175
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   ID        450176 non-null  uint32
 1   District  450176 non-null  uint8 
dtypes: uint32(1), uint8(1)
memory usage: 2.1 MB


In [21]:
# Memory usage of old and new Dataframe
print(df[['ID', 'District']].memory_usage() / btomb)
print(conv_df.memory_usage() / btomb)
print("\nWe saved more than half of the space here")

Index       0.000122
ID          3.434570
District    3.434570
dtype: float64
Index       0.000122
ID          1.717285
District    0.429321
dtype: float64

We saved more than half of the space here


In [22]:
#Just check that we deal with the same data
sim = conv_df['District'].astype("float") != df.District
print("Nr. of discrepancies: %d"%sim.sum())

Nr. of discrepancies: 0


In [23]:
df.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location'],
      dtype='object')

In [24]:
print(df['X Coordinate'][df['X Coordinate'].notna()].min(), df['X Coordinate'][df['X Coordinate'].notna()].max())

0.0 1205117.0


In [25]:
np.finfo('float16')

finfo(resolution=0.001, min=-6.55040e+04, max=6.55040e+04, dtype=float16)

In [26]:
(df['X Coordinate'][df['X Coordinate'].notna()].astype('int').astype('float64') != df['X Coordinate'][df['X Coordinate'].notna()]).sum()

0

Seems like none of the floats are really float

### Objects are still there. What can we do with them?

We could use categories where it is appropriate. If the number of unique elements are very low, let's say less then 100 and (we can name them easily), then it really makes sense.

In [27]:
print(len(df['Primary Type'].unique()), df['Primary Type'].notna().sum())

35 450175


Looks like 'Primary type' is a column where categories would make sense.

In [28]:
conv_df['Primary Type'] = df['Primary Type'].astype('category')

In [29]:
print(df[['ID', 'District', 'Primary Type']].memory_usage() / btomb)
print(conv_df.memory_usage() / btomb)

Index           0.000122
ID              3.434570
District        3.434570
Primary Type    3.434570
dtype: float64
Index           0.000122
ID              1.717285
District        0.429321
Primary Type    0.430603
dtype: float64


In [30]:
conv_df.head()

,ID,District,Primary Type
0,11034701,4,DECEPTIVE PRACTICE
1,11227287,22,CRIM SEXUAL ASSAULT
2,11227583,8,BURGLARY
3,11227293,3,THEFT
4,11227634,1,CRIM SEXUAL ASSAULT


Caveat!

In [31]:
ages = np.array([10, 15, 13, 12, 23, 25, 28, 59, 60])
pd.cut(ages, bins=5)

[(9.95, 20.0], (9.95, 20.0], (9.95, 20.0], (9.95, 20.0], (20.0, 30.0], (20.0, 30.0], (20.0, 30.0], (50.0, 60.0], (50.0, 60.0]]
Categories (5, interval[float64, right]): [(9.95, 20.0] < (20.0, 30.0] < (30.0, 40.0] < (40.0, 50.0] < (50.0, 60.0]]

There are codes referring to the category

In [32]:
conv_df['Primary Type'].cat.codes

0          9
1          5
2          3
3         32
4          5
          ..
450171    24
450172    17
450173     8
450174    18
450175    -1
Length: 450176, dtype: int8

In [33]:
df[['Date']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450176 entries, 0 to 450175
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Date    450176 non-null  object
dtypes: object(1)
memory usage: 3.4+ MB


In [34]:
df[['Date']].head()

,Date
0,01/01/2001 11:00:00 AM
1,10/08/2017 03:00:00 AM
2,03/28/2017 02:00:00 PM
3,09/09/2017 08:17:00 PM
4,08/26/2017 10:00:00 AM


In [35]:
#pd.to_datetime(df[['Date']],format='%m/%d/%Y %I:%M:%S %p', errors='raise')
conv_df['Date'] = df[['Date']].apply(lambda x: pd.to_datetime(x, format='%m/%d/%Y %I:%M:%S %p'))
#pd.to_datetime(df[['Date']].iloc[:10])

#df["DateTime"] = pd.to_datetime(df["DateTime"],errors="coerce").dt.strftime("%d-%m-%Y %H:%M:%S")

In [36]:
print(df[['ID', 'District', 'Primary Type', 'Date']].memory_usage() / btomb)
print(conv_df.memory_usage() / btomb)

Index           0.000122
ID              3.434570
District        3.434570
Primary Type    3.434570
Date            3.434570
dtype: float64
Index           0.000122
ID              1.717285
District        0.429321
Primary Type    0.430603
Date            3.434570
dtype: float64


In [37]:
conv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450176 entries, 0 to 450175
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   ID            450176 non-null  uint32        
 1   District      450176 non-null  uint8         
 2   Primary Type  450175 non-null  category      
 3   Date          450176 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](1), uint32(1), uint8(1)
memory usage: 6.0 MB


In [38]:
cs = conv_df[['ID', 'District', 'Primary Type']].memory_usage().sum()
ds = df[['ID', 'District', 'Primary Type']].memory_usage().sum()
print("%d %%"%((cs/ds)*100))


25 %


### What was not covered here

* [Sparse column representation](https://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html): very few relevant values 
* [Loading data in chunks can shrink memory usage](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html): 
  * function we run on each chunk can run independently
  * Dask provides an API that emulates Pandas, while implementing chunking and parallelization transparently.
* [Lossy compression](https://pythonspeed.com/articles/pandas-reduce-memory-lossy/)